In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from utils.environment_specific import is_local_development
from utils.dataset import load_public_dataset, load_full_private_df

In [2]:
datasets = {
    "grambeddings": "grambeddings",
    "kaggle_binary": "kaggle_binary",
    "kaggle_multiple": "kaggle_multiple",
    "mendeley": "mendeley",
}
dataset_names = datasets.keys()
raw_folder = "./data/raw"
target_folder = "./data/processed"

In [3]:
all_dfs = []

for dataset_name, _ in datasets.items():
    dataset_path = os.path.join(raw_folder, dataset_name, "dataset.csv")
    df = pd.read_csv(dataset_path)
    df = df[["url", "label"]].copy()
    df["dataset"] = dataset_name  # use the dict key as the dataset identifier
    all_dfs.append(df)

if not is_local_development():
    datasets["private_data"] = "private_data"
    

all_datasets = pd.concat(all_dfs, ignore_index=True)

In [4]:
dataset_count = len(all_datasets["dataset"].unique())
assert dataset_count == 4, "Not all datasets were added to all_datasets"

In [5]:
def get_dataset(dataset_name):
    return all_datasets[all_datasets['dataset'] == dataset_name]

# High level statistics

In [6]:
display(all_datasets.info())
display(all_datasets.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3507328 entries, 0 to 3507327
Data columns (total 3 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   url      object
 1   label    object
 2   dataset  object
dtypes: object(3)
memory usage: 80.3+ MB


None

,url,label,dataset
0,https://blog.sockpuppet.us/,benign,grambeddings
1,https://blog.apiki.com/seguranca/,benign,grambeddings
2,http://autoecole-lauriston.com/a/T0RVd056QXlNe...,malicious,grambeddings
3,http://chinpay.site/index.html?hgcFSE@E$Z*DFcG...,malicious,grambeddings
4,http://www.firstfivenebraska.org/blog/article/...,benign,grambeddings


In [7]:
for name in dataset_names:
    df_subset = all_datasets[all_datasets["dataset"] == name]
    count = df_subset["label"].value_counts()
    percent = (count / len(df_subset) * 100).round(2)
    percent_of_all = (count / len(all_datasets) * 100).round(2)

    summary = pd.DataFrame({"count": count, "%_dataset": percent, "%_joined_dataset": percent_of_all})

    total_row = pd.DataFrame(
        {"count": [len(df_subset)], "%_dataset": [100.0], "%_joined_dataset": [len(df_subset) / len(all_datasets) * 100]},
        index=["TOTAL"],
    )

    summary = pd.concat([summary, total_row])

    print(f"=== Dataset: {name} ===")
    print(summary.round(2))
    print()

=== Dataset: grambeddings ===
            count  %_dataset  %_joined_dataset
benign     400001       50.0             11.40
malicious  399995       50.0             11.40
TOTAL      799996      100.0             22.81

=== Dataset: kaggle_binary ===
            count  %_dataset  %_joined_dataset
benign     316252       50.0              9.02
malicious  316251       50.0              9.02
TOTAL      632503      100.0             18.03

=== Dataset: kaggle_multiple ===
             count  %_dataset  %_joined_dataset
benign      342651      66.81              9.77
defacement   76252      14.87              2.17
phishing     75135      14.65              2.14
malware      18857       3.68              0.54
TOTAL       512895     100.00             14.62

=== Dataset: mendeley ===
             count  %_dataset  %_joined_dataset
benign     1526619      97.74             43.53
malicious    35315       2.26              1.01
TOTAL      1561934     100.00             44.53



# Removal of data
- Everything related to removing records form any dataset is in this section

## Remove rows with conflicting labels

- When we manually inspect generated csv files, we see that most of the conflict invovle Mendeley dataset. 
    - Also most amount of duplicates comes from here - this is crawled by https://developers.google.com/safe-browsing so it was possibly run multiple times and maybe made a mistake
    - It either first classified as bening and then corrected to malignant or the other way
- There is no temporal information in data, so we do not know which way the error originated - safest would be to remove them altogether

In [8]:
def get_conflicting_label_indices(df):
    conflicting_urls = df.groupby("url")["label"].nunique()
    conflicting_urls = conflicting_urls[conflicting_urls > 1].index
    indices = df[df["url"].isin(conflicting_urls)].index

    repetition_counts = df[df["url"].isin(conflicting_urls)].groupby("url").size()
    grouped_repetition = repetition_counts.value_counts().sort_index()

    print(f"Number of records to remove: {len(indices)}")
    print(f"Number of unique URLs to remove: {len(conflicting_urls)}")
    print("Number of conflicting URLs grouped by how many times they are repeated:")
    print(grouped_repetition)

    return indices

### Conflicting multi-class labels

- we first check whether there are any conflicts within the multiple datset
- there are none

In [9]:
kaggle_multiple = all_datasets[all_datasets['dataset'] == datasets['kaggle_multiple']]
to_remove_indices = get_conflicting_label_indices(kaggle_multiple)
# nothing to remove
del kaggle_multiple

Number of records to remove: 0
Number of unique URLs to remove: 0
Number of conflicting URLs grouped by how many times they are repeated:
Series([], Name: count, dtype: int64)


- To do this correctly, we must rename phishing, defacement and malware labels from kaggle_multiple to malicious (kaggle_binary is created from this partially and this would cause label conflicts)
- Mendeley dataset causes a lot of these misclassifications because of the nature of its collection
    - Most is checked by https://developers.google.com/safe-browsing so it was possibly run multiple times and maybe made a mistake
    - It either first classified as bening and then corrected to malignant or the other way
- There is no temporal information in data, so we do not know which way the error originated - safest would be to remove them altogether

In [10]:
def get_binary_named_dataset(df):
    relabelled_df = df.copy()
    relabelled_df["label"] = np.where(df["label"] == "benign", "benign", "malicious")
    return relabelled_df

relabelled_df = get_binary_named_dataset(all_datasets)

conflicting_indices = get_conflicting_label_indices(relabelled_df)

conflicting_records = relabelled_df.loc[conflicting_indices].sort_values(by="url")
print("Dataset presence in conflicting records:")
print(conflicting_records["dataset"].value_counts())
print("Examples:")
print(conflicting_records.head(16))

before = len(all_datasets)
all_datasets = all_datasets.drop(index=conflicting_indices).reset_index(drop=True)
assert len(relabelled_df) - len(all_datasets) == len(conflicting_records)

del relabelled_df

Number of records to remove: 215
Number of unique URLs to remove: 105
Number of conflicting URLs grouped by how many times they are repeated:
2    100
3      5
Name: count, dtype: int64
Dataset presence in conflicting records:
dataset
mendeley           195
kaggle_binary       12
kaggle_multiple      5
grambeddings         3
Name: count, dtype: int64
Examples:
                                           url      label          dataset
845123             ebookstore.sony.com/reader/  malicious    kaggle_binary
1712839            ebookstore.sony.com/reader/     benign  kaggle_multiple
1540836           en.wikipedia.org/wiki/E-book     benign  kaggle_multiple
1415800           en.wikipedia.org/wiki/E-book  malicious    kaggle_binary
1534475  groups.yahoo.com/group/Band-in-a-Box/     benign  kaggle_multiple
1090247  groups.yahoo.com/group/Band-in-a-Box/  malicious    kaggle_binary
2259601       http://allyourtrekarebelongto.us  malicious         mendeley
2235206       http://allyourtrekarebe

## Remove defacement URLs
- Defacement is a type of attack, which cannot be detected from just URL string (see thesis explanation)
- Its inclusion in dataset would force the model to overfit or learn wrong information
- --> Remove all defacement URLs from the kaggle_multiple dataset. 
- Since kaggle_binary is partially created from kaggle_multiple (see thesis), we also remove these from kaggle_binary

- First, look at how much content is shared between kaggle_binary and kaggle_multiple

In [11]:
df_binary = all_datasets[all_datasets["dataset"] == datasets["kaggle_binary"]]
df_multiple = all_datasets[all_datasets["dataset"] == datasets["kaggle_multiple"]]

urls_binary = set(df_binary["url"])
urls_multiple = set(df_multiple["url"])
shared_urls = urls_binary.intersection(urls_multiple)
print(f"Total shared URLs: {len(shared_urls)}")
print(f"% of kaggle_binary also present in kaggle_multiple: {len(shared_urls) / len(df_binary) * 100:.2f}%")
print(f"% of kaggle_multiple also present in kaggle_binary: {len(shared_urls) / len(df_multiple) * 100:.2f}%\n")

shared_df_binary = df_binary[df_binary["url"].isin(shared_urls)]
shared_df_multiple = df_multiple[df_multiple["url"].isin(shared_urls)]


def print_label_sharing_stats(name, full_df, shared_df):
    full_counts = full_df["label"].value_counts()
    shared_counts = shared_df["label"].value_counts()

    summary = pd.DataFrame(
        {
            "total_count": full_counts,
            "shared_count": shared_counts,
        }
    ).fillna(0)

    summary["%shared_of_label"] = (summary["shared_count"] / summary["total_count"] * 100).round(2)

    print(f"=== {name} ===")
    print(summary)
    print()


print_label_sharing_stats(datasets["kaggle_binary"], df_binary, shared_df_binary)
print_label_sharing_stats(datasets["kaggle_multiple"], df_multiple, shared_df_multiple)

Total shared URLs: 170168
% of kaggle_binary also present in kaggle_multiple: 26.90%
% of kaggle_multiple also present in kaggle_binary: 33.18%

=== kaggle_binary ===
           total_count  shared_count  %shared_of_label
label                                                 
benign          316252           0.0              0.00
malicious       316239      170168.0             53.81

=== kaggle_multiple ===
            total_count  shared_count  %shared_of_label
label                                                  
benign           342646           0.0              0.00
defacement        76252       76230.0             99.97
malware           18857       18853.0             99.98
phishing          75135       75085.0             99.93



In [12]:
defacement_urls = all_datasets.loc[
    (all_datasets["dataset"] == "kaggle_multiple") & (all_datasets["label"] == "defacement"), "url"
]

all_datasets_filtered = all_datasets[~all_datasets["url"].isin(defacement_urls)].reset_index(drop=True)

print(
    f"Removed {len(all_datasets) - len(all_datasets_filtered)} records associated with 'defacement' from 'kaggle_multiple' and duplicates."
)
all_datasets = all_datasets_filtered
del all_datasets_filtered

Removed 152482 records associated with 'defacement' from 'kaggle_multiple' and duplicates.


## Within dataset duplication
- we see that only for mendeley dataset, there are duplicates for non-processed URLs

In [13]:
relabelled_df = get_binary_named_dataset(all_datasets.copy())

def pct(n, d):
    return 0 if d == 0 else n / d * 100

result = []
for dataset_name in dataset_names:
    df = relabelled_df[relabelled_df['dataset'] == dataset_name]
    records_count = len(df)
    uniq_df = df.drop_duplicates(subset=["url"])
    unique_count = len(uniq_df)

    duplicates_cnt = records_count - unique_count
    duplicates_pct = pct(duplicates_cnt, records_count)

    mal_df = df[df["label"] == "malicious"]
    ben_df = df[df["label"] == "benign"]

    result.append(
        {
            "dataset": dataset_name,
            "records_count": records_count,
            "duplicates_count": duplicates_cnt,
            "duplicates %": duplicates_pct,
            "unique_count": unique_count,
            "unique %": pct(unique_count, records_count),
            "benign %": pct(len(ben_df), records_count),
            "malicious %": pct(len(mal_df), records_count),
            "unique_benign %": pct(len(ben_df.drop_duplicates(subset=["url"])), len(ben_df)),
            "unique_malicious %": pct(len(mal_df.drop_duplicates(subset=["url"])), len(mal_df)),
        }
    )

all_datasets.drop_duplicates(subset=["dataset", "url"], inplace=True)

In [14]:
print(f"Removed {len(relabelled_df) - len(all_datasets)}")
display(pd.DataFrame(result))
del relabelled_df

Removed 47637


,dataset,records_count,duplicates_count,duplicates %,unique_count,unique %,benign %,malicious %,unique_benign %,unique_malicious %
0,grambeddings,799993,0,0.000000,799993,100.000000,50.000188,49.999812,100.000000,100.000000
1,kaggle_binary,556261,0,0.000000,556261,100.000000,56.853168,43.146832,100.000000,100.000000
2,kaggle_multiple,436638,0,0.000000,436638,100.000000,78.473701,21.526299,100.000000,100.000000
3,mendeley,1561739,47637,3.050254,1514102,96.949746,97.744694,2.255306,96.892599,99.426495


# Folds

In [15]:
import heapq
from utils.url_features import gini_coefficient
from utils.dataset import analyze_folds

def assign_folds(dataset, domain_stats: pd.DataFrame, n_folds: int = 5, seed: int = 42) -> pd.DataFrame:
    """
    Greedy assignment that balances total_records first, malicious count second.
    Complexity: O(D log F) where D = #domains, F = #folds.
    """
    np.random.seed(seed)
    shuffled = domain_stats.sample(frac=1, random_state=seed)
    # (total - first criterion, malicious_count - second criterion, i)
    heap = [(0, 0, i) for i in range(n_folds)]
    heapq.heapify(heap)

    domain, fold = [], []
    for d, total, malicious in shuffled[["sld", "total_records", "malicious"]].to_numpy():
        total_f, malicious_f, fid = heapq.heappop(heap)
        domain.append(d)
        fold.append(fid)
        heapq.heappush(heap, (total_f + total, malicious_f + malicious, fid))

    assigned = pd.DataFrame({'sld': domain, 'fold': fold})
    assigned = assigned.merge(domain_stats[["sld", "total_records"]], on="sld", how="left")
    gini_per_fold = assigned.groupby("fold")["total_records"].apply(lambda arr: gini_coefficient(arr.values))

    # outlier-heavy first
    new_order = (
        gini_per_fold.sort_values(ascending=False)  
        .reset_index()
        .assign(new_fold=lambda d: d.index)
    )
    remap = dict(zip(new_order["fold"], new_order["new_fold"]))
    assigned["fold"] = assigned["fold"].map(remap)

    grambeddigns_folds = dataset.merge(assigned, on='sld', how='left')

    return grambeddigns_folds

In [16]:
from utils.dataset import get_domain_stats
from utils.url_features import get_sld

In [17]:
all_datasets['sld'] = all_datasets['url'].apply(get_sld)

In [18]:
all_datasets_copy = []
for dataset_name in dataset_names:
    print(f"=== {dataset_name} ===")
    df = get_dataset(dataset_name)
    df_domain_stats = df
    if dataset_name == datasets["kaggle_multiple"]:
        df_domain_stats = get_binary_named_dataset(df)
    dataset = assign_folds(df, domain_stats=get_domain_stats(df_domain_stats))
    display(analyze_folds(dataset))
    dataset["dataset"] = dataset_name
    all_datasets_copy.append(dataset)

all_datasets = pd.concat(all_datasets_copy, ignore_index=True)

=== grambeddings ===


,fold,total,unique_domains,benign,benign_pct,malicious,malicious_pct,top_1,top_2,top_3
0,0,159998,67929,70075,43.797422,89923,56.202578,000webhostapp (12.1%),blogspot (2.8%),duckdns (1.2%)
1,1,159998,77389,80077,50.048751,79921,49.951249,appspot (6.3%),bandcamp (1.3%),podbean (1.1%)
2,2,159999,77665,85930,53.706586,74069,46.293414,alibaba (5.3%),platino (1.2%),netlify (0.9%)
3,3,160000,82604,82363,51.476875,77637,48.523125,buap (1.8%),weebly (0.8%),libsyn (0.5%)
4,4,159998,82855,81553,50.971262,78445,49.028738,webcindario (1.7%),69.167.151.209 (1.1%),justns (1.0%)


=== kaggle_binary ===


,fold,total,unique_domains,benign,benign_pct,malicious,malicious_pct,top_1,top_2,top_3
0,0,111245,31791,68194,61.300733,43051,38.699267,wikipedia (10.4%),youtube (7.8%),myspace (2.6%)
1,1,111245,33123,61248,55.056856,49997,44.943144,blogspot (5.6%),linkedin (3.6%),ietf (2.9%)
2,2,111245,34219,64168,57.681694,47077,42.318306,facebook (7.0%),amazon (4.6%),yahoo (4.0%)
3,3,111245,36308,62273,55.978246,48972,44.021754,imdb (3.1%),answers (2.2%),manta (1.9%)
4,4,111281,37131,60369,54.249153,50912,45.750847,mylife (2.5%),000webhostapp (1.9%),123people (1.7%)


=== kaggle_multiple ===


,fold,total,unique_domains,benign,benign_pct,malware,malware_pct,phishing,phishing_pct,top_1,top_2,top_3
0,0,87293,20706,68789,78.802424,3126,3.581043,15378,17.616533,wikipedia (12.4%),youtube (7.9%),facebook (7.6%)
1,1,87328,24458,69716,79.832356,3182,3.643734,14430,16.523910,yahoo (5.3%),amazon (4.8%),imdb (3.2%)
2,2,87293,26008,67645,77.491895,5297,6.068070,14351,16.440035,linkedin (4.6%),myspace (2.7%),mixh (2.7%)
3,3,87393,26260,68249,78.094355,2964,3.391576,16180,18.514069,blogspot (7.7%),answers (2.3%),wn (1.8%)
4,4,87331,29516,68247,78.147508,4288,4.910055,14796,16.942437,ancestry (2.3%),wordpress (1.7%),googlegroups (1.4%)


=== mendeley ===


,fold,total,unique_domains,benign,benign_pct,malicious,malicious_pct,top_1,top_2,top_3
0,0,304794,141180,297977,97.763407,6817,2.236593,geocities (17.4%),imdb (3.3%),yahoo (3.2%)
1,1,302327,153900,295983,97.901610,6344,2.098390,angelfire (6.8%),newadvent (3.9%),gamespot (1.3%)
2,2,302327,174591,294585,97.439197,7742,2.560803,tripod (7.2%),ietf (1.1%),webring (0.8%)
3,3,302327,180016,295265,97.664119,7062,2.335881,freewebs (1.8%),blogspot (1.6%),gamespy (0.9%)
4,4,302327,181844,295272,97.666434,7055,2.333566,wikipedia (2.1%),homestead (1.1%),cstv (0.7%)


# Store results

In [19]:
def store_result(dataset_subset):
    assert {"url", "label", "fold", "dataset"}.issubset(dataset_subset.columns), "Missing required columns"
    
    dataset_name = dataset_subset["dataset"].unique()
    assert len(dataset_name) == 1, "dataset_subset must contain only one dataset"
    dataset_name = dataset_name[0]

    train_df = dataset_subset[dataset_subset["fold"].isin([0, 1, 2, 3])][["url", "label", "fold"]].reset_index(drop=True)
    test_df = dataset_subset[dataset_subset["fold"] == 4][["url", "label", "fold"]].reset_index(drop=True)

    ds_folder = os.path.join("data/processed", dataset_name)
    os.makedirs(ds_folder, exist_ok=True)

    train_df.to_csv(os.path.join(ds_folder, "train.csv"), index=False)
    test_df.to_csv(os.path.join(ds_folder, "test.csv"), index=False)

    print(f"Stored: {dataset_name}")

In [20]:
for ds_name in all_datasets["dataset"].unique():
    subset = all_datasets[all_datasets["dataset"] == ds_name]
    store_result(subset)

Stored: grambeddings
Stored: kaggle_binary
Stored: kaggle_multiple
Stored: mendeley


# Create joined dataset

In [21]:
joined_dataset = all_datasets[all_datasets['dataset'] != datasets['kaggle_multiple']]
joined_dataset = joined_dataset.drop(columns=["fold"])

- there are no conflicts now (removed previously), so we can just drop duplicates

In [22]:
joined_dataset = joined_dataset.drop_duplicates(subset=["url"])

In [23]:
joined_dataset.info()
joined_dataset["label"].value_counts()

<class 'pandas.core.frame.DataFrame'>
Index: 2862060 entries, 0 to 3306993
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   url            object
 1   label          object
 2   dataset        object
 3   sld            object
 4   total_records  int64 
dtypes: int64(1), object(4)
memory usage: 131.0+ MB


label
benign       2194899
malicious     667161
Name: count, dtype: int64

In [24]:
joined_dataset = assign_folds(joined_dataset, domain_stats=get_domain_stats(joined_dataset))
display(analyze_folds(joined_dataset))

,fold,total,unique_domains,benign,benign_pct,malicious,malicious_pct,top_1,top_2,top_3
0,0,572662,245422,444298,77.584683,128364,22.415317,geocities (9.3%),appspot (1.8%),alibaba (1.5%)
1,1,572351,255202,449272,78.495888,123079,21.504112,tripod (4.1%),wikipedia (3.1%),blogspot (2.7%)
2,2,572349,264155,444623,77.683896,127726,22.316104,angelfire (3.9%),yahoo (2.5%),imdb (2.3%)
3,3,572349,274267,414583,72.435350,157766,27.564650,000webhostapp (3.7%),ietf (1.1%),sourceforge (0.8%)
4,4,572349,281081,442123,77.247099,130226,22.752901,newadvent (2.0%),google (0.6%),about (0.6%)


- we can see that the old datasets are pretty well distributed across new folds

In [25]:
fold_counts = []
for fold in sorted(joined_dataset["fold"].unique()):
    vc = joined_dataset.loc[joined_dataset["fold"] == fold, "dataset"].value_counts(normalize=True)
    for label, proportion in vc.items():
        fold_counts.append({"fold": fold, "dataset": label, "proportion": proportion})

df_counts = pd.DataFrame(fold_counts).sort_values(["fold", "dataset"]).reset_index(drop=True)
print(df_counts)

    fold        dataset  proportion
0      0   grambeddings    0.279215
1      0  kaggle_binary    0.176958
2      0       mendeley    0.543827
3      1   grambeddings    0.265073
4      1  kaggle_binary    0.206842
5      1       mendeley    0.528085
6      2   grambeddings    0.265888
7      2  kaggle_binary    0.202794
8      2       mendeley    0.531317
9      3   grambeddings    0.310961
10     3  kaggle_binary    0.193427
11     3       mendeley    0.495612
12     4   grambeddings    0.276445
13     4  kaggle_binary    0.177942
14     4       mendeley    0.545613


In [26]:
joined_dataset['dataset'] = 'joined'
store_result(joined_dataset)

Stored: joined


- script that compares the output versions of datasets
- useful when making changes to this code (check that the content remains the same)

In [ ]:
# import os
# import pandas as pd

# base1 = "./data/processed"
# base2 = "./data/processed_copy"

# subfolders = [
#     name for name in os.listdir(base1)
#     if os.path.isdir(os.path.join(base1, name))
# ]

# for subfolder in sorted(subfolders):
#     path1 = os.path.join(base1, subfolder)
#     path2 = os.path.join(base2, subfolder)

#     if not os.path.isdir(path2):
#         print(f"Missing in copy: {subfolder}")
#         continue

#     files1 = set(os.listdir(path1))
#     files2 = set(os.listdir(path2))

#     common_files = files1 & files2

#     for filename in sorted(common_files):
#         file1 = os.path.join(path1, filename)
#         file2 = os.path.join(path2, filename)

#         try:
#             with open(file1, "r", encoding="utf-8") as f1, open(file2, "r", encoding="utf-8") as f2:
#                 lines1 = f1.readlines()
#                 lines2 = f2.readlines()

#             if lines1 == lines2:
#                 print(f"{subfolder}/{filename}: Identical")
#             else:
#                 print(f"{subfolder}/{filename}: Content differs (line-by-line)")
#         except Exception as e:
#             print(f"{subfolder}/{filename}: Failed to read - {e}")

grambeddings/test.csv: OK
grambeddings/train.csv: OK
joined/test.csv: OK
joined/train.csv: OK
kaggle_binary/test.csv: OK
kaggle_binary/train.csv: OK
kaggle_multiple/test.csv: OK
kaggle_multiple/train.csv: OK
mendeley/test.csv: OK
mendeley/train.csv: OK
